<a href="https://colab.research.google.com/github/yashAICodeHUB/Ml_project_end_to_end/blob/main/llama_3.1_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('hello')

hello


In [ ]:
# Meta LLama 3:B model size is areound 15 gb so i first i use quantization process, where we quantize model

In [2]:
 # !pip install -r requirements.txt
# already install all requirement libraries

# ==================================================
# we also put huggingFace token to config.json file

In [21]:
# Importing and configuring transformer libraries
import json
import torch    # for setting up datatype for quantization
from transformers import (AutoTokenizer,  # from which we load tokenizer from llama 3 model
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline
                          )

In [4]:
# we need to configure transformer library
config_data = json.load(open('config.json'))   # load will convert json into dictionary and stored in variable
HF_TOKEN= config_data['HF_TOKEN']

In [6]:
# HF_TOKEN

In [7]:
model_name = "meta-llama/Llama-3.1-8B"

In [9]:
# Quantization configure
# The LLm that we are using are very big in size, so this process shrink the model
# It basically change the presicion of weight, we know all the deep learning model having paramater of weight
# So this weight is in higher presicion of 32bi. so we have ability to use 16bit or 8bit, so that why we called Quantization

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,        # so we load model into 4bit
    bnb_4bit_use_double_quant=True,  # here we define that we want to quantize the model but does not want to lose lot of information
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [12]:
# loading the tokenizer and model

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token

# when we pass sentence to tokenizer, ideally all token should have same length, but if some sentence are short in that case we pass extra
# space to it, when we pass eos_token it mark the sentence that this is end of sentence


In [14]:
# above we load tokenizer, now we are going to load LLM model

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto', # Device map decide where to load our model, on gpu or cpu, if we select auto, it automatically detect gpu and load
                       # model in that gpu

    quantization_config=bnb_config,  # here if we not set this paramter, it will not quantize the model
    token=HF_TOKEN
)


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [22]:
# now lets create pipeline

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128

)


Device set to use cuda:0


In [24]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text =sequences[0]['generated_text']
  return gen_text

In [34]:
prompt = "what is machine learning?"

In [35]:
llama_response = get_response(prompt)

In [36]:
llama_response

'what is machine learning? | definition, history, applications, and more\nMachine learning is a subset of artificial intelligence (AI) that focuses on the development of algorithms and models that can learn and improve from data without explicit programming. It is a field of computer science that deals with the design and development of systems that can learn from data, identify patterns, and make predictions or decisions based on those patterns.\nMachine learning is used in a wide range of applications, including image and speech recognition, natural language processing, and recommendation systems. It is also used in fields such as healthcare, finance, and retail to improve decision-making and automate processes.\nThere are several types of'

In [37]:
print(llama_response[len(prompt):])

 | definition, history, applications, and more
Machine learning is a subset of artificial intelligence (AI) that focuses on the development of algorithms and models that can learn and improve from data without explicit programming. It is a field of computer science that deals with the design and development of systems that can learn from data, identify patterns, and make predictions or decisions based on those patterns.
Machine learning is used in a wide range of applications, including image and speech recognition, natural language processing, and recommendation systems. It is also used in fields such as healthcare, finance, and retail to improve decision-making and automate processes.
There are several types of
